In [1]:
import simi

import pandas as pd
from sentence_transformers import evaluation, losses, models, InputExample, SentenceTransformer
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

MODEL = SentenceTransformer('all-mpnet-base-v2')

RANDOM_STATE = 1

2024-01-06 18:46:12.580585: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
df = pd.read_csv("anchor-arxiv-dataset.csv", index_col=0)
df.sample(5)

,title,abstract,categories,doi,a,p,n
40585,On Higher Graph Manifolds,In this short note we introduce higher graph...,"('math.GT', 'math.DG')",10.1093/imrn/rnx133,"In the case without any pure pieces, we show t...",In the case when the pure pieces are hyperboli...,We give a complete list of hyperbolic two-bri...
88808,Dynamic Learning of Sequential Choice Bandit P...,Motivated by the observation that overexposu...,"('cs.LG', 'stat.ML')",NaN,"For the online problem, we propose an algorith...",Motivated by the observation that overexposur...,Experiments with synthetic and real data are p...
78084,Tropical Support Vector Machines: Evaluations ...,Support Vector Machines (SVMs) are one of th...,"('cs.LG', 'math.CO', 'math.ST', 'stat.TH')",NaN,While the generalization error bounds attained...,Support Vector Machines (SVMs) are one of the...,"As an application, we improve and extend to th..."
64756,Efficient Modular Arithmetic for SIMD Devices,This paper describes several new improvement...,"('cs.CR', 'cs.PF', 'math.OC')",NaN,This new results allow building personal compu...,This paper describes several new improvements...,"In addition, the paper discusses a comprehensi..."
15680,Weighted Sobolev Spaces on Metric Measure Spaces,We investigate weighted Sobolev spaces on me...,"('math.AP', 'math.FA', 'math.MG')",NaN,"Denoting by $\rho$ the weight function, we com...",Under mild assumptions on the metric measure s...,"Moreover, inconditional uniqueness is obtained..."


In [3]:
X_treval, X_test = train_test_split(df, train_size=0.9, random_state=RANDOM_STATE)
X_train, X_eval = train_test_split(X_treval, train_size=0.888889, random_state=RANDOM_STATE)
print("train:", len(X_train), "eval:", len(X_eval), "test:", len(X_test))

train: 73760 eval: 9221 test: 9221


In [4]:
# prepare evaluation data
eval_examples = X_eval.reset_index(drop=True).apply(lambda r: InputExample(texts=[r["a"], r["p"], r["n"]]), axis=1)
evaluator = evaluation.TripletEvaluator.from_input_examples(eval_examples)

In [5]:
# prepare re-training: training data, loss
retrain_examples = X_train.reset_index(drop=True).apply(lambda r: InputExample(texts=[r["a"], r["p"], r["n"]]), axis=1)
retrain_dataloader = DataLoader(retrain_examples, shuffle=True, batch_size=16)
retrain_loss = losses.TripletLoss(model=MODEL, triplet_margin=5) # TODO: triplet_margin: hyperparameter to optimize

In [6]:
%%time
# finetune model
MODEL.fit(train_objectives=[(retrain_dataloader, retrain_loss)], evaluator=evaluator, epochs=10,
        output_path="sbert+retrain_anchor_arxiv") 

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4610 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4610 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4610 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4610 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/4610 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4610 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4610 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4610 [00:00<?, ?it/s]

CPU times: user 20h 44min 47s, sys: 20min 31s, total: 21h 5min 18s
Wall time: 5h 5min 16s


In [7]:
%%time
eval_res = MODEL.evaluate(evaluator)
eval_res

CPU times: user 9min 9s, sys: 13.7 s, total: 9min 22s
Wall time: 56.3 s


0.6901637566424466